In [1]:
# Reactive programming layer for Julia
using Rocket 
# Core package for Constrained Bethe Free Energy minimsation with Factor graphs and message passing
using ReactiveMP 
# High-level user friendly probabilistic model and constraints specification language for ReactiveMP
using GraphPPL
# Optionally include Distributions.jl
using Distributions

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1342
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1342


### General syntax for model creation

We use `@model` macro from `GraphPPL.jl` package to create a probabilistic model $p(s|y)$ and also specify extra constraints on variational family of distributions $\mathcal{Q}$.
Below there is a simple example of general syntax for model creation. In this tutorial we do not cover all possible way to create models or extra features of `GraphPPL.jl` and we refer a reader to the documentation for more rigorous explanations and examples.

In [35]:
# `@model` macro accepts a regular Julia function
@model function test_model1(s_mean, s_precision)
    
    # We use `randomvar` function to create 
    # random variables in our model
    s = randomvar()
    
    # `tilde` expression creates a functional dependencies
    # between variables in our model and can be read as 
    # `sampled from`
    s ~ GaussianMeanPrecision(s_mean, s_precision)
    
    # We use `datavar` function to create 
    # observed data variables in our models
    # We also need to specify the type of our data 
    # In this example it is `Float64`
    y = datavar(Float64)
    
    y ~ GaussianMeanPrecision(s, 1.0)
    
    return s, y
end

test_model1 (generic function with 3 methods)

`@model` macro creates a function with the same name and with the same set of input arguments as the original function (`test_model1(s_mean, s_precision)` in this example). However, the return value is modified in such a way to contain a reference to the model object as first value and user specified variables in a form of a tuple as second value.

In [36]:
model, (s, y) = test_model1(0.0, 1.0);

Later on we can examine our model structure with the help of some utility functions such as: 
- `getnodes()`: returns an array of factor nodes in a correposning factor graph
- `getrandom()`: returns an array of random variable in the model
- `getdata()`: returns an array of data inputs in the model
- `getconstant()`: return an array of constant values in the model

In [37]:
getnodes(model)

2-element Vector{ReactiveMP.AbstractFactorNode}:
 FactorNode:
 form            : NormalMeanPrecision
 sdtype          : Stochastic()
 interfaces      : (Interface(out,Marginalisation()), Interface(μ,Marginalisation()), Interface(τ,Marginalisation()))
 factorisation   : ((1, 2, 3),)
 local marginals : (:out_μ_τ,)
 metadata        : nothing
 pipeline        : FactorNodePipeline(functional_dependencies = DefaultFunctionalDependencies(), extra_stages = EmptyPipelineStage()

 FactorNode:
 form            : NormalMeanPrecision
 sdtype          : Stochastic()
 interfaces      : (Interface(out,Marginalisation()), Interface(μ,Marginalisation()), Interface(τ,Marginalisation()))
 factorisation   : ((1, 2, 3),)
 local marginals : (:out_μ_τ,)
 metadata        : nothing
 pipeline        : FactorNodePipeline(functional_dependencies = DefaultFunctionalDependencies(), extra_stages = EmptyPipelineStage()


In [38]:
getrandom(model) .|> name

1-element Vector{Symbol}:
 :s

In [39]:
getdata(model) .|> name

1-element Vector{Symbol}:
 :y

In [40]:
getconstant(model) .|> getconst

3-element Vector{Float64}:
 0.0
 1.0
 1.0

It is also possible to control flow statements in model specification such as `if` or `for` blocks

In [41]:
@model function test_model2(n)
    
    if n <= 1
        error("`n` argument must be greater than one.")
    end
    
    # `randomvar(n)` creates a dense sequence of 
    # random variables
    s = randomvar(n)
    
    # `datavar(Float64, n)` creates a dense sequence of 
    # observed data variables of type `Float64`
    y = datavar(Float64, n)
    
    s[1] ~ GaussianMeanPrecision(0.0, 0.1)
    y[1] ~ GaussianMeanPrecision(s[1], 1.0)
    
    for i in 2:n
        s[i] ~ GaussianMeanPrecision(s[i - 1], 1.0)
        y[i] ~ GaussianMeanPrecision(s[i], 1.0)
    end
    
    return s, y
end

test_model2 (generic function with 1 method)

In [44]:
model, (s, y) = test_model2(10);

In [53]:
# An amount of factor nodes in generated Factor Graph
getnodes(model) |> length

20

In [54]:
# An amount of random variables
getrandom(model) |> length

10

In [55]:
# An amount of data inputs
getdata(model) |> length

10

In [56]:
# An amount of constant values
getconstant(model) |> length

21

It is also possible to use complex expression inside functional dependencies expressions

```julia
y ~ NormalMeanPrecision(2.0 * (s + 1.0), 1.0)
```

`~` operator automatically creates a random variable if none was created before with the same name and errors if this name already exists

```julia
# s = randomvar() here is optional
# `~` creates random variables automatically
s ~ NormalMeanPrecision(0.0, 1.0)
```

An error example:

In [84]:
@model function error_model1()
    s = 1.0
    s ~ NormalMeanPrecision(0.0, 1.0)
end

LoadError: LoadError: Invalid name 's' for new random variable. 's' was already initialized with '=' operator before.
in expression starting at /Users/bvdmitri/.julia/dev/GraphPPL/src/GraphPPL.jl:161

By default `GraphPPL.jl` creates a new references for constants (literals like `0.0` or `1.0`) in a model. In some situtations it maybe not efficient especially if this constants represent some matrices. `GraphPPL.jl` will create a new copy of some constant matrix in a model every time it uses it. However it is possible to use `constavar()` function to create and reuse constant in the model specification syntax

```julia
# Creates constant reference in a model with a prespecified value
c = constvar(0.0)
```

An example:

In [74]:
@model function test_model5(dim::Int, n::Int, A::Matrix, P::Matrix, Q::Matrix)
    
    s = randomvar(n)
    
    y = datavar(Vector{Float64}, n)
    
    # Here we create constant references
    # for constant matrices in our model 
    # to make inference a little bit more efficient
    cA = constvar(A)
    cP = constvar(P)
    cQ = constvar(Q)
    
    s[1] ~ MvNormalMeanCovariance(zeros(dim), cP)
    y[1] ~ MvNormalMeanCovariance(s[1], cQ)
    
    for i in 2:n
        s[i] ~ MvNormalMeanCovariance(cA * s[i - 1], cP)
        y[i] ~ MvNormalMeanCovariance(s[i], cQ)
    end
    
    return s, y
end

test_model5 (generic function with 1 method)

### Factorisation constraints for variational family of distributions $\mathcal{Q}$

On a very high-level, ReactiveMP.jl is aimed to solve the Constrained Bethe Free Energy minimisation problem. For this task we often need to specify extra factorisation on variatonal family of distributions $q \in \mathcal{Q}$. For this purpose `@model` macro supports optional `where { ... }` clauses for `~` expressions in a model specification.

In [ ]:
@model function test_model6(n)
    τ ~ GammaShapeRate(1.0, 1.0) 
    μ ~ NormalMeanVariance(0.0, 100.0)
    
    y = datavar(Float64, n)
    
    for i in 1:n
        # Here we assume a mean-field assumption on our 
        # variational family of distributions
        y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i])q(μ)q(τ) }
    end
    
    return μ, τ, y
end

There are several options to specify the mean-field factorisation constraint. 

```julia
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i])q(μ)q(τ) } # With names from model specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(out)q(mean)q(precision) } # With names from node specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = MeanField() } # With alias name
```

It is also possible to use local structured factorisation:

```julia
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i], μ)q(τ) } # With names from model specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(out, mean)q(precision) } # With names from node specification
```

In [ ]:
length(getdata(model))